In [137]:
import pandas as pd
from scipy import sparse
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

In [138]:
y_train = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_train_sem_Fasting_Blood_Glucose.csv')

In [139]:
y_train.columns

Index(['Diabetes'], dtype='object')

In [140]:
y_test = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_test_sem_Fasting_Blood_Glucose.csv')

In [141]:
x_train = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_train_sem_Fasting_Blood_Glucose.csv')

In [142]:
x_test = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_test_sem_Fasting_Blood_Glucose.csv')

In [143]:
x_train

,onehot__Sex_Female,onehot__Sex_Male,onehot__Ethnicity_Asian,onehot__Ethnicity_Black,onehot__Ethnicity_Hispanic,onehot__Ethnicity_White,onehot__Physical_Activity_Level_High,onehot__Physical_Activity_Level_Low,onehot__Physical_Activity_Level_Moderate,onehot__Alcohol_Consumption_Heavy,...,scaler__Blood_Pressure_Systolic,scaler__Blood_Pressure_Diastolic,scaler__Cholesterol_Total,scaler__Cholesterol_HDL,scaler__Cholesterol_LDL,scaler__GGT,scaler__Serum_Urate,scaler__Dietary_Intake_Calories,scaler__Family_History_of_Diabetes,scaler__Previous_Gestational_Diabetes
0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,-0.198051,0.604274,1.177843,-0.706220,-1.068551,0.333800,0.136893,0.490099,0.989917,0.963853
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.685005,-0.032495,0.176666,1.032038,1.171279,1.379244,-0.274857,1.329937,0.989917,0.963853
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.427802,0.893715,-1.676330,0.977073,-1.276783,-1.598923,1.029020,-1.048441,-1.010185,-1.037503
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.608145,-0.379823,-1.431286,-0.561938,-0.342408,0.900886,0.480019,-1.703822,-1.010185,0.963853
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.263105,-1.595472,-0.864186,-0.946691,1.440913,-1.471618,1.509396,1.099366,0.989917,0.963853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.531286,-0.379823,1.257190,1.011426,1.585074,-1.074272,-0.480733,-0.313407,0.989917,-1.037503
7496,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.889786,1.646260,-1.011212,0.166344,-0.833622,0.121624,-0.343482,-1.105734,-1.010185,0.963853
7497,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.504662,-0.785039,1.653927,-0.974173,-0.943077,-0.889101,-0.412108,0.744426,-1.010185,0.963853
7498,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,-1.350943,0.777938,0.104319,1.382438,-1.501033,-0.626775,-0.617983,1.651340,0.989917,0.963853


In [144]:
type(x_train)

pandas.core.frame.DataFrame

In [145]:
x_train.columns

Index(['onehot__Sex_Female', 'onehot__Sex_Male', 'onehot__Ethnicity_Asian',
       'onehot__Ethnicity_Black', 'onehot__Ethnicity_Hispanic',
       'onehot__Ethnicity_White', 'onehot__Physical_Activity_Level_High',
       'onehot__Physical_Activity_Level_Low',
       'onehot__Physical_Activity_Level_Moderate',
       'onehot__Alcohol_Consumption_Heavy',
       'onehot__Alcohol_Consumption_Moderate',
       'onehot__Smoking_Status_Current', 'onehot__Smoking_Status_Former',
       'onehot__Smoking_Status_Never', 'scaler__Age', 'scaler__BMI',
       'scaler__Waist_Circumference', 'scaler__Fasting_Blood_Glucose',
       'scaler__Blood_Pressure_Systolic', 'scaler__Blood_Pressure_Diastolic',
       'scaler__Cholesterol_Total', 'scaler__Cholesterol_HDL',
       'scaler__Cholesterol_LDL', 'scaler__GGT', 'scaler__Serum_Urate',
       'scaler__Dietary_Intake_Calories', 'scaler__Family_History_of_Diabetes',
       'scaler__Previous_Gestational_Diabetes'],
      dtype='object')

#### Grid trabalha com dados completos, não precisa separar em treino e teste, pois ele já faz isso com a validação cruzada

In [146]:
x_geral = pd.concat([x_train,x_test], axis=0)

In [147]:
y_geral = pd.concat([y_train,y_test], axis=0)

#### Bernoulli

In [148]:
from sklearn.naive_bayes import BernoulliNB

In [149]:
bernoulli = BernoulliNB()
bernoulli.fit(x_train, y_train)
previsao_bernoulli = bernoulli.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [150]:
acuracia_bernoulli = accuracy_score(previsao_bernoulli, y_test)
print(f"Acurácia bernoulli: {acuracia_bernoulli}")

Acurácia bernoulli: 0.8392


In [151]:
print(classification_report(y_test,previsao_bernoulli))

              precision    recall  f1-score   support

           0       0.90      0.81      0.85      1412
           1       0.78      0.88      0.83      1088

    accuracy                           0.84      2500
   macro avg       0.84      0.84      0.84      2500
weighted avg       0.85      0.84      0.84      2500



In [152]:
print(confusion_matrix(y_test,previsao_bernoulli))

[[1141  271]
 [ 131  957]]


#### Árvores

In [153]:
parametros = {
    "max_depth":[500, 600, 650],
    "min_samples_split":[2,5,10],
    "min_samples_leaf":[1,5,10],
    "criterion":["gini", "entropy"]
}

In [154]:
from sklearn.tree import DecisionTreeClassifier

In [155]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#### Grid trabalha com dataset completo

In [156]:
randomized = RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42), param_distributions=parametros, verbose=True, n_jobs=4, cv=3, n_iter=20)
randomized.fit(x_train,y_train)
melhores_parametros = randomized.best_params_
melhor_resultado = randomized.best_score_


Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [157]:
print(melhor_resultado)

0.8064


In [158]:
print(melhores_parametros)

{'min_samples_split': 5, 'min_samples_leaf': 10, 'max_depth': 650, 'criterion': 'gini'}


In [159]:
arbol = DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=1, max_depth=650, criterion='gini')
arbol.fit(x_train, y_train)
previsao_arbol = arbol.predict(x_test)

In [160]:
cross_arbol1 = cross_val_score(arbol,x_train,y_train, cv=5)
print(cross_arbol1.mean())

0.7944000000000001


In [161]:
print(accuracy_score(y_test,previsao_arbol))

0.8


In [162]:
print(classification_report(y_test,previsao_arbol))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83      1412
           1       0.78      0.74      0.76      1088

    accuracy                           0.80      2500
   macro avg       0.80      0.79      0.80      2500
weighted avg       0.80      0.80      0.80      2500



In [163]:
print(confusion_matrix(y_test,previsao_arbol))

[[1190  222]
 [ 278  810]]


#### Profundidade da árvore

In [164]:
arbol.get_depth()

42

#### Numero de folhas

In [165]:
arbol.get_n_leaves()

np.int64(757)

In [166]:
arbol.tree_

##### Serve para ver as features mais importantes, mas ele perdeu o nome das features

In [167]:
importances = pd.DataFrame({
    "features":x_train.columns,
    "importancia":arbol.feature_importances_
}).sort_values(by="importancia", ascending=False)

In [168]:
importances.head() # Maior fator de diabetes é o índice glicêmico em jejum

,features,importancia
17,scaler__Fasting_Blood_Glucose,0.618529
22,scaler__Cholesterol_LDL,0.037799
16,scaler__Waist_Circumference,0.036029
15,scaler__BMI,0.034696
25,scaler__Dietary_Intake_Calories,0.033024


#### Regressão logística

In [169]:
from sklearn.linear_model import LogisticRegression

In [170]:
parametros = {
    'C':[1.0,2.0,3.0,4.0],
    'penalty':['l2','elasticnet', 'None'],
    'tol':[0.001, 0.0001, 0.00001]
}

In [171]:
randomized_regression = RandomizedSearchCV(estimator=LogisticRegression(), param_distributions=parametros, n_jobs=4, verbose=True)
randomized_regression.fit(x_train, y_train)
melhores_parametros = randomized_regression.best_params_
melhor_resultado = randomized_regression.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\skle

In [172]:
print(melhor_resultado)

0.7934666666666668


In [173]:
print(melhores_parametros)

{'tol': 0.001, 'penalty': 'l2', 'C': 2.0}


In [174]:
logistic = LogisticRegression(max_iter=1000)
logistic.fit(x_train, y_train)
previsao_logistic = logistic.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [175]:
print(accuracy_score(previsao_logistic,y_test))

0.7992


In [176]:
print(classification_report(y_test,previsao_logistic))

              precision    recall  f1-score   support

           0       0.82      0.83      0.82      1412
           1       0.77      0.76      0.77      1088

    accuracy                           0.80      2500
   macro avg       0.80      0.79      0.80      2500
weighted avg       0.80      0.80      0.80      2500



In [177]:
cross_logistic = cross_val_score(logistic, x_train, y_train, cv=5)
cross_logistic.mean()

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

np.float64(0.7932)

In [178]:
print(confusion_matrix(previsao_logistic,y_test))

[[1171  261]
 [ 241  827]]


#### Random

In [179]:
from sklearn.ensemble import RandomForestClassifier

#### Verificar se o random forest está overfitando ou underfitando, verificar os parâmetros default

In [180]:
from scipy.stats import randint

In [181]:
parametros = {
    "n_estimators": randint(100,301), # no randint coloca o maior e menor
    "min_samples_split": randint(10,30),
    "min_samples_leaf": randint(10,30),
    "max_features": ['log2','sqrt'],
    "max_depth": randint(10,20)
}

In [182]:
randomized_random = RandomizedSearchCV(estimator=RandomForestClassifier(), param_distributions=parametros, n_jobs=4, verbose=True)
randomized_random.fit(x_train,y_train)
melhores_parametros = randomized_random.best_params_
melhor_resultado = randomized_random.best_score_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [183]:
print(melhor_resultado)

0.8713333333333333


In [184]:
print(melhores_parametros)

{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 22, 'min_samples_split': 22, 'n_estimators': 124}


In [185]:
random = RandomForestClassifier(verbose=True, n_jobs=4)
random.fit(x_train,y_train)
previsao_random = random.predict(x_test)
previsao_ingenua_random = random.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [186]:
print(accuracy_score(previsao_random,y_test))

0.8796


In [187]:
print(classification_report(y_test,previsao_random))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88      1412
           1       0.78      1.00      0.88      1088

    accuracy                           0.88      2500
   macro avg       0.89      0.89      0.88      2500
weighted avg       0.90      0.88      0.88      2500



In [188]:
print(confusion_matrix(previsao_random,y_test))

[[1115    4]
 [ 297 1084]]


In [189]:
print(accuracy_score(previsao_ingenua_random,y_train))

1.0


In [190]:
cross_random = cross_val_score(random, x_train,y_train,cv=5)
print(f"Cross random: {cross_random.mean()}")

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(

Cross random: 0.8704000000000001


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [191]:
print(random.max_depth)

None


#### Número de árvores

In [192]:
print(random.n_estimators)

100


#### KNN

In [193]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(x_train,y_train)
previsao_KNN = KNN.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [194]:
print(accuracy_score(previsao_KNN,y_test))

0.7544


In [195]:
print(classification_report(y_test,previsao_KNN))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78      1412
           1       0.71      0.73      0.72      1088

    accuracy                           0.75      2500
   macro avg       0.75      0.75      0.75      2500
weighted avg       0.76      0.75      0.75      2500



In [196]:
print(confusion_matrix(previsao_KNN,y_test))

[[1090  292]
 [ 322  796]]


##### SVM

In [197]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

In [198]:
parametros = {
    "C":[1,2,4,5,6],
    "kernel":['poly','rbf','sigmoid','linear']
}

In [199]:
randomized_SVM = RandomizedSearchCV(estimator=SVC(), param_distributions=parametros, n_jobs=4, verbose=True)
randomized_SVM.fit(x_train,y_train)
melhores_parametros = randomized_SVM.best_params_
melhor_resultado = randomized_SVM.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [200]:
print(melhor_resultado)

0.8289333333333333


In [201]:
print(melhores_parametros)

{'kernel': 'rbf', 'C': 2}


In [202]:
SVM = SVC()
SVM.fit(x_train,y_train)
previsao_SVM = SVM.predict(x_test)
previsao_ingenua = SVM.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [203]:
print(accuracy_score(previsao_ingenua,y_train))

0.8757333333333334


In [204]:
print(classification_report(y_test, previsao_SVM))

              precision    recall  f1-score   support

           0       0.92      0.80      0.85      1412
           1       0.78      0.91      0.84      1088

    accuracy                           0.85      2500
   macro avg       0.85      0.85      0.85      2500
weighted avg       0.86      0.85      0.85      2500



In [205]:
print(confusion_matrix(previsao_SVM,y_test))

[[1126   97]
 [ 286  991]]


In [206]:
cross_SVM = cross_val_score(SVM,x_train,y_train,cv=5)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

In [207]:
print(cross_SVM.mean())

0.8324


In [208]:
print(accuracy_score(previsao_SVM,y_test))

0.8468


##### xgboost

In [209]:
#### Ocorreu underfitting

In [210]:
from xgboost import XGBClassifier

In [211]:
xgb_params = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

In [212]:
randomized_xg = RandomizedSearchCV(estimator=XGBClassifier(), param_distributions=xgb_params, n_jobs=4, verbose=True)
randomized_xg.fit(x_train,y_train)
melhores_parametros = randomized_xg.best_params_
melhor_resultado = randomized_xg.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [213]:
print(melhor_resultado)

0.8708


In [214]:
print(melhores_parametros)

{'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 1.0}


In [215]:
XG = XGBClassifier(n_estimator = 200, min_child_weight=1, max_depth=9, learning_rate=0.2, gamma=0.2, colsample_bytree = 0.8, subsample = 1.0)
XG.fit(x_train,y_train)
cross_xg = cross_val_score(XG,x_train,y_train,cv=5)
previsao_XG = XG.predict(x_test)
previsao_ingenua = XG.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:54:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:54:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:54:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:54:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_esti

In [216]:
print(accuracy_score(previsao_ingenua,y_train)) 

0.9993333333333333


In [217]:
print(cross_xg.mean())

0.8610666666666666


In [218]:
print(accuracy_score(previsao_XG,y_test))

0.8736


In [219]:
print(classification_report(y_test, previsao_XG))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1412
           1       0.79      0.97      0.87      1088

    accuracy                           0.87      2500
   macro avg       0.88      0.89      0.87      2500
weighted avg       0.89      0.87      0.87      2500



In [220]:
print(confusion_matrix(previsao_XG,y_test))

[[1124   28]
 [ 288 1060]]
